In [1]:
!git clone https://github.com/mymusise/ChatGLM-Tuning.git
%cd  ChatGLM-Tuning
!pip install -r requirements.txt 

Cloning into 'ChatGLM-Tuning'...
remote: Enumerating objects: 163, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 163 (delta 74), reused 65 (delta 61), pack-reused 51
Receiving objects: 100% (163/163), 8.03 MiB | 17.13 MiB/s, done.
Resolving deltas: 100% (86/86), done.
/content/ChatGLM-Tuning
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-3n8huk8f
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-3n8huk8f
  Resolved https://github.com/huggingface/peft.git to commit 632997d1fb776c3cf05d8c2537ac9a98a7ce9435
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━

In [2]:
# 将此处修改为instruction数据
data_path = "./data/alpaca_data.json"
save_path = "/content/drive/MyDrive/Colab Notebooks/output/"

In [3]:
from transformers import AutoTokenizer, AutoModel, TrainingArguments, AutoConfig
import torch
import torch.nn as nn
from peft import get_peft_model, LoraConfig, TaskType


class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)
    
model = AutoModel.from_pretrained(
        "THUDM/chatglm-6b", load_in_8bit=True, trust_remote_code=True, device_map="auto"
    )
model.gradient_checkpointing_enable()
model.enable_input_require_grads()
model.is_parallelizable = True
model.model_parallel = True
model.lm_head = CastOutputToFloat(model.lm_head)
model.config.use_cache = (
    False  # silence the warnings. Please re-enable for inference!
)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('http'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_sa

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.


Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


# Test before Fine-Tune

In [5]:
# 这部分有些问题，有时候会无输出，结果都为0
# 实测，当此处输出有问题时，fine-tune也会失败
from cover_alpaca2jsonl import format_example
import json
from random import sample

data = json.load(open(data_path))
instruction_samples = data[:5]
answer_samples = []

with torch.no_grad():
  for idx, item in enumerate(instruction_samples):
    feature = format_example(item)
    input_text = feature["context"]
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    out = model.generate(input_ids, max_length=150)
    answer = tokenizer.decode(out[0])
    print(answer)
    answer_samples.append(answer)
    item['infer_answer'] = answer
    print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1374: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Instruction: Give three tips for staying healthy.
Answer: 1.  Intake regular exercise.
2.   Eat a balanced diet with a variety of fruits, vegetables, lean protein, and whole grains.
3.  Get enough sleep each night.

### 1.Answer:
 1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. 
2. Exercise regularly to keep your body active and strong. 
3. Get enough sleep and maintain a consistent sleep schedule. 


Instruction: What are the three primary colors?
Answer: The three primary colors of painting are red, blue, and green, which are also the primary colors of the color system used in the RGB color space. This system uses these colors to create other colors by mixing them together.

### 2.Answer:
 The three primary colors are red, blue, and yellow. 


Instruction: Describe the structure of an atom.
Answer: An原子 is the smallest unit of matter that contains all the other elements in the universe. It is made up of a central core of positive charge, surrounded by 

# Fine-Tune



In [6]:
import datasets
from dataclasses import dataclass, field
import transformers
from tqdm import tqdm
from tokenize_dataset_rows import preprocess

@dataclass
class FineTuneArgs:
    json_path: str = field(default="./data/alpaca_data.json")
    max_seq_length: int = field(default=384)
    skip_overlength: bool = field(default=False)
    model_path: str = field(default="output")
    lora_rank: int = field(default=8)

In [7]:
finetune_args = FineTuneArgs(
    json_path = data_path,
    max_seq_length = 384,
    skip_overlength = False,
    model_path = save_path,
    lora_rank = 8
)

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=False,
    r=finetune_args.lora_rank,
    lora_alpha=32, lora_dropout=0.1,
)
model = get_peft_model(model, peft_config)
model.is_parallelizable = True
model.model_parallel = True

In [8]:
def read_json(path, max_seq_length, skip_overlength=False):
    model_name = "THUDM/chatglm-6b"
    tokenizer = transformers.AutoTokenizer.from_pretrained(
        model_name, trust_remote_code=True)
    config = transformers.AutoConfig.from_pretrained(
        model_name, trust_remote_code=True, device_map='auto')
    with open(path, "r") as f:
        examples = json.load(f)
        for example in tqdm(examples, "waiting..."):
            example = format_example(example)
            feature = preprocess(tokenizer, config, example, max_seq_length)
            if skip_overlength and len(feature["input_ids"]) > max_seq_length:
                continue
            feature["input_ids"] = feature["input_ids"][:max_seq_length]
            yield feature

In [9]:
# TODO 重写这一块的tqdm
from datasets import disable_caching
disable_caching()
dataset = datasets.Dataset.from_generator(
        lambda: read_json(finetune_args.json_path, finetune_args.max_seq_length, finetune_args.skip_overlength)
    )

train_num = 500

mini_train_dataset = datasets.Dataset.from_dict(dataset[:train_num])

Generating train split: 0 examples [00:00, ? examples/s]

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.

waiting...: 100%|██████████| 52002/52002 [00:45<00:00, 1141.04it/s]


Dataset generator downloaded and prepared to /root/.cache/huggingface/datasets/generator/default-c8b1e06cfc671de5/0.0.0. Subsequent calls will reuse this data.


In [10]:
from transformers import Trainer, HfArgumentParser


def data_collator(features: list) -> dict:
    len_ids = [len(feature["input_ids"]) for feature in features]
    longest = max(len_ids)
    input_ids = []
    labels_list = []
    for ids_l, feature in sorted(zip(len_ids, features), key=lambda x: -x[0]):
        ids = feature["input_ids"]
        seq_len = feature["seq_len"]
        labels = (
            [-100] * (seq_len - 1) + ids[(seq_len - 1) :] + [-100] * (longest - ids_l)
        )
        ids = ids + [tokenizer.pad_token_id] * (longest - ids_l)
        _ids = torch.LongTensor(ids)
        labels_list.append(torch.LongTensor(labels))
        input_ids.append(_ids)
    input_ids = torch.stack(input_ids)
    labels = torch.stack(labels_list)
    return {
        "input_ids": input_ids,
        "labels": labels,
    }

class ModifiedTrainer(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False):
        return model(
            input_ids=inputs["input_ids"],
            labels=inputs["labels"],
        ).loss

In [11]:
torch.autocast("cuda")
training_args = TrainingArguments(
    save_path,
    fp16 = True,
    gradient_accumulation_steps=1,
    per_device_train_batch_size = 1,
    learning_rate = 1e-4,
    max_steps=1500,
    logging_steps=50,
    remove_unused_columns=False,
    seed=0,
    data_seed=0,
    group_by_length=False,
)


trainer = ModifiedTrainer(
    model=model,
    train_dataset=mini_train_dataset,
    args=training_args,
    data_collator=data_collator,
)

with torch.autocast("cuda"): 
    trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
50,2.775000
100,2.138200
150,2.045100
200,1.802400
250,1.933100
300,1.737200
350,1.758300
400,1.609700
450,1.442800
500,1.695600


/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


In [12]:
model.save_pretrained(save_path)